In [2]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='1234',
                             db='sakila',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [3]:
# check connection

connection

In [4]:
# read database tables wtih panda

import pandas as pd
db_tables = pd.read_sql_query("show tables from sakila", connection)
db_tables

,Tables_in_sakila
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


In [5]:
# describe tables in the database

for table in db_tables["Tables_in_sakila"]:
    output = pd.read_sql_query(f'describe {table}', connection).tail(1)
    print(table)
    print(output)

actor
         Field       Type Null Key            Default  \
3  last_update  timestamp   NO      CURRENT_TIMESTAMP   

                                           Extra  
3  DEFAULT_GENERATED on update CURRENT_TIMESTAMP  
actor_info
       Field  Type Null Key Default Extra
3  film_info  text  YES        None      
address
         Field       Type Null Key            Default  \
8  last_update  timestamp   NO      CURRENT_TIMESTAMP   

                                           Extra  
8  DEFAULT_GENERATED on update CURRENT_TIMESTAMP  
category
         Field       Type Null Key            Default  \
2  last_update  timestamp   NO      CURRENT_TIMESTAMP   

                                           Extra  
2  DEFAULT_GENERATED on update CURRENT_TIMESTAMP  
city
         Field       Type Null Key            Default  \
3  last_update  timestamp   NO      CURRENT_TIMESTAMP   

                                           Extra  
3  DEFAULT_GENERATED on update CURRENT_TIMESTAMP  
country
 

### making queries with pandas

In [6]:
# turning table to a dataframe

payment_df = pd.read_sql_query("select * from payment", connection)
payment_df

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76.0,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573.0,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185.0,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422.0,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476.0,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30
...,...,...,...,...,...,...,...
16044,16045,599,1,14599.0,4.99,2005-08-21 17:43:42,2006-02-15 22:24:12
16045,16046,599,1,14719.0,1.99,2005-08-21 21:41:57,2006-02-15 22:24:12
16046,16047,599,2,15590.0,8.99,2005-08-23 06:09:44,2006-02-15 22:24:12
16047,16048,599,2,15719.0,2.99,2005-08-23 11:08:46,2006-02-15 22:24:13


In [7]:
# finding max payment amount

max_payment = pd.read_sql_query("select max(amount) from payment", connection)
max_payment

,max(amount)
0,11.99


In [8]:
# customer id's who made max payment

customer_max_payment =  pd.read_sql_query("select customer_id from payment where amount = (select max(amount) from payment)", connection)
customer_max_payment

,customer_id
0,13
1,116
2,195
3,196
4,204
5,237
6,305
7,362
8,591
9,592


In [9]:
# average and total values of payment

payment_info =  pd.read_sql_query("select avg(amount) as Average_payment, sum(amount) as Total_payment from payment", connection)
payment_info

,Average_payment,Total_payment
0,4.200667,67416.51


In [10]:
# to find top 10 customers payment amount above the average

payment_info =  pd.read_sql_query("select * from payment where amount > (select avg(amount) from payment) order by amount desc limit 10", connection)
payment_info

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,3146,116,2,14763,11.99,2005-08-21 23:34:00,2006-02-15 22:13:08
1,8272,305,1,2166,11.99,2005-06-17 23:51:21,2006-02-15 22:15:47
2,15821,591,2,4383,11.99,2005-07-07 20:45:51,2006-02-15 22:23:18
3,6409,237,2,11479,11.99,2005-08-02 22:18:13,2006-02-15 22:14:38
4,15850,592,1,3973,11.99,2005-07-06 22:58:31,2006-02-15 22:23:25
5,9803,362,1,14759,11.99,2005-08-21 23:28:58,2006-02-15 22:16:57
6,5550,204,2,15415,11.99,2005-08-22 23:48:56,2006-02-15 22:14:07
7,5280,195,2,16040,11.99,2005-08-23 22:19:33,2006-02-15 22:14:00
8,5281,196,2,106,11.99,2005-05-25 18:18:19,2006-02-15 22:14:00
9,342,13,2,8831,11.99,2005-07-29 22:37:41,2006-02-15 22:12:31


In [11]:
# join customer and address table and show 10 rows

customer_addresses =  pd.read_sql_query("SELECT * FROM customer join address on address.address_id = customer.address_id limit 10", connection)
customer_addresses

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,address_id,address,address2,district,city_id,postal_code,phone,location,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,5,1913 Hanoi Way,,Nagasaki,463,35200,28303384290,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00(\xd17\x...,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20,6,1121 Loja Avenue,,California,449,17886,838635286649,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00<\x94W\x...,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20,7,692 Joliet Street,,Attika,38,83579,448477190408,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x07oY\x...,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20,8,1566 Inegl Manor,,Mandalay,349,53561,705814003527,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x06$\x0...,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,9,53 Idfu Parkway,,Nantou,361,42399,10655648674,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x1f\x81...,2006-02-15 04:57:20
5,6,2,JENNIFER,DAVIS,JENNIFER.DAVIS@sakilacustomer.org,10,1,2006-02-14 22:04:36,2006-02-15 04:57:20,10,1795 Santiago de Compostela Way,,Texas,295,18743,860452626434,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00PP/\x9d{...,2006-02-15 04:57:20
6,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,11,900 Santiago de Compostela Parkway,,Central Serbia,280,93896,716571220373,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00...,2006-02-15 04:57:20
7,8,2,SUSAN,WILSON,SUSAN.WILSON@sakilacustomer.org,12,1,2006-02-14 22:04:36,2006-02-15 04:57:20,12,478 Joliet Way,,Hamilton,200,77948,657282285970,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xdc\x84...,2006-02-15 04:57:20
8,9,2,MARGARET,MOORE,MARGARET.MOORE@sakilacustomer.org,13,1,2006-02-14 22:04:36,2006-02-15 04:57:20,13,613 Korolev Drive,,Masqat,329,45844,380657522649,"b""\x00\x00\x00\x00\x01\x01\x00\x00\x00\x01\x02...",2006-02-15 04:57:20
9,10,1,DOROTHY,TAYLOR,DOROTHY.TAYLOR@sakilacustomer.org,14,1,2006-02-14 22:04:36,2006-02-15 04:57:20,14,1531 Sal Drive,,Esfahan,162,53628,648856936185,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xde\tQ\...,2006-02-15 04:57:20
